In [165]:
import re, time
import json
import requests


#hash this as we'll be using it a lot
def get_page(url,params={}):
    return session_requests.get(
        url,
        params=params,
        headers = dict(referer=url))

session_requests = requests.session()
login_url = 'https://www.vinted.fr/member/general/login?ref_url='
tt = session_requests.get(login_url).content.decode('utf-8')
pp = re.compile('name="csrf-token" content="(.*?)"')
mm = pp.search(tt)

from vinted_creds import creds
payload = creds
payload["authenticity_token"] =  mm.group(1)

result = session_requests.post( 'https://www.vinted.fr/member/general/login.json',
                               json=json.dumps(payload),
                               headers = dict(referer=login_url)
                              )

result

<Response [422]>

In [177]:
def get_items(id_='20263980'):
    r = get_page('https://www.vinted.fr/api/v2/users/%s/'%id_)
    user_data = r.json()['user']
    
    num_items = user_data['item_count']
    if num_items == 0: 
        return user_data, []

    url = 'https://www.vinted.fr/api/v2/users/%s/items'%id_
    stuff = []
    
    for k in range(1, num_items//48 + 2):
        print('page', k)
        r = get_page(url, 
                    params={'page': k, 'per_page': 48})
        stuff.extend( r.json()['items'])
        
    return user_data, stuff


id_='32135520'
id_='20263980'
uu, stuff = get_items(id_=id_)
    

page 1
page 2
page 3
page 4
page 5
page 6


In [167]:
sum( [ float(x['price_numeric']) for x in stuff ] )

200.0

1


In [178]:
my_keys = ['id', 'view_count', 'created_at_ts', 'price_numeric', 
          'title', 'description','path']

col_names = ['id', 'views', 'created', 'price', 
           'title', 'description','path']

rows = [[ art[x] for x in my_keys ] for art in stuff ]
cols = list(zip(* rows))

cols = {x : y for x,y in zip(col_names, cols)}
cols['created'] = [x.split('T')[0] for x in cols['created']]
cols['price'] =[float(x) for x in cols['price']]

today = datetime.strftime(datetime.now(), '%Y-%m-%d')
with open('%s_%s.json'%(id_, today),'w') as fp:
    json.dump(cols, fp)
    
df_art = pd.DataFrame( cols)


In [179]:
df_art.sort_values('price', ascending=False)

,id,views,created,price,title,description,path
256,356389866,4,2019-11-23,20.0,Jean Teddy smith gris fille,jamais porté avec étiquette acheté 79 euros\nt...,/enfants/pantalons-jeans-slim/356389866-jean-t...
255,356394179,13,2019-11-23,12.0,Jean Teddy Smith bleu,jamais porté avec étiquette \nacheté 79 euros\...,/enfants/pantalons-jeans-slim/356394179-jean-t...
32,378630167,39,2019-12-27,12.0,polo lacoste rose framboise femme 34,très bon état,/femmes/autres-hauts/378630167-polo-lacoste-ro...
251,356415276,6,2019-11-23,10.0,Pantalon noir velours fille repetto,jamais porté \ntaille très bien \ntrès bonne q...,/enfants/pantalons-jeans-slim/356415276-pantal...
261,255167306,23,2019-07-11,10.0,pantalon blanc esprit,jamais porté très bien taillé,/femmes/pantalons-skinny/255167306-pantalon-bl...
6,380590063,4,2019-12-30,10.0,pantalon blanc esprit femme 36 neuf,jamais porté \ntaille parfaitement,/femmes/jeans-skinny/380590063-pantalon-blanc-...
207,369823925,58,2019-12-11,10.0,veste petit bateau marine fille,veste très bonne qualité/ acheté 135 euros/ pa...,/enfants/vetements-dexterieur-vestes/369823925...
228,356594365,58,2019-11-23,10.0,manteau bleu marine petit bateau fille laine (...,porté quelques \nce manteau taille parfaitemen...,/enfants/vetements-dexterieur-manteaux/3565943...
193,377582711,3,2019-12-25,7.5,Le désespoir des singes et autres bagatelles,prix d’achat: 21€\nneuf,/maison/biographies-and-memoires/377582711-le-...
204,377547013,4,2019-12-25,7.0,Inferno - Dan Brown,prix d’achat: 22€90\nneuf,/maison/policiers-and-thrillers/377547013-infe...


In [143]:
from datetime import datetime
def str2date(ss):
    date = ss.split('+')[0]
    return  datetime.strptime(date, '%Y-%m-%dT%H:%M:%S')

def date2offset(ss):
    start = str2date('2018-12-02T15:53:11+01:00')
    
    return  (datetime.now() - str2date(ss) ).days

rows[0]

[384239510,
 6,
 '2020-01-05T11:57:38+01:00',
 '5.0',
 'haut zara noir S',
 'tres peu porté ',
 '/femmes/tops-courts/384239510-haut-zara-noir-s']

In [13]:
with open('classement.txt','w') as fp:
    fp.write( '\n'.join(['%d %d %s'%x for x in xx]) )

In [105]:
r = get_page('https://www.vinted.fr/member/general/followers/20263980')
px = re.compile('membres/(\d+)-(.*?)"')
friends = list( set(px.findall(r.content.decode('utf-8'))) )

In [154]:
friends


[('24794428', 'mariehvz'),
 ('15350990', 'evaamw'),
 ('32977581', 'doudetaurelien'),
 ('13862386', 'simojo10'),
 ('12651286', 'locean21'),
 ('31290234', 'bibou0087'),
 ('16147769', 'iannelaure'),
 ('12271904', 'smarchetti'),
 ('32135520', 'bertrandclavel'),
 ('32147659', 'michelinjeanmarc'),
 ('31678660', 'brisbane63'),
 ('28779534', 'mthiollier'),
 ('24794404', 'ceeliaa19'),
 ('20263980', 'erinmcsh'),
 ('10559472', 'adeleonillon'),
 ('15226661', 'karimusnebili'),
 ('26278824', 'colinegvs'),
 ('32423957', 'paul11040'),
 ('32032265', 'manon-rchr38'),
 ('32359937', 'marie-gabvort')]

In [140]:
import time

keys = [ 'login', 'gender', 'city',
'item_count', 'given_item_count', 
'created_at', 'last_loged_on_ts' ]

data = []

for id_,name  in friends:
    print(name)
    r = get_page('https://www.vinted.fr/api/v2/users/%s/'%id_)
    xx = r.json()['user']
    data.append([ xx[k] for k in keys ])
    time.sleep(3)
    
    

mariehvz
evaamw
doudetaurelien
simojo10
locean21
bibou0087
iannelaure
smarchetti
bertrandclavel
michelinjeanmarc
brisbane63
mthiollier
ceeliaa19
erinmcsh
adeleonillon
karimusnebili
colinegvs
paul11040
manon-rchr38
marie-gabvort


In [141]:
cols = dict( zip( keys, list(zip(*data)) ) )

for f, t in zip((0,1), ['created_at', 'last_loged_on_ts' ]):
    cols[t] = [ x.split('T')[f] for x in cols [t] ]

In [83]:
import pandas as pd

In [142]:
pd.DataFrame(cols)

,login,gender,city,item_count,given_item_count,created_at,last_loged_on_ts
0,marie.hvz,F,Grenoble,2,5,2019-05-05,20:52:36+01:00
1,evaa.mw,F,Échirolles,38,19,2018-01-29,20:48:20+01:00
2,doudetaurelien,O,,0,0,2020-01-07,07:18:22+01:00
3,simojo10,F,,20,16,2017-09-18,21:22:41+01:00
4,locean21,F,Versailles,29,45,2017-04-08,20:27:53+01:00
5,bibou0087,F,Béziers,435,19,2019-11-13,07:56:44+01:00
6,i.annelaure,M,Grenoble,0,0,2018-03-20,09:58:24+01:00
7,smarchetti,F,,26,4,2017-02-14,21:24:22+01:00
8,bertrand_clavel,M,Grenoble,47,31,2019-12-08,07:05:35+01:00
9,michelin.jeanmarc,,Grenoble,0,0,2019-12-08,15:38:03+01:00
